In [2]:
import numpy as np
# import modin.pandas as pd
import pandas as pd
from IPython.display import Image
from pandas_profiling import ProfileReport
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import (KNNImputer, MissingIndicator,
                            SimpleImputer, IterativeImputer)
from sklearn.linear_model import BayesianRidge, Lasso, LinearRegression, Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import (GridSearchCV, cross_validate,
                                     train_test_split)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (LabelEncoder, MinMaxScaler, OneHotEncoder,
                                   OrdinalEncoder, StandardScaler)
from xgboost import XGBRegressor


In [3]:

final_test = pd.read_csv('data/test.csv')
train_init = pd.read_csv('data/train.csv')
train_init = train_init.drop(columns=["Id"])


labels = train_init["SalePrice"]
features = train_init.drop(columns=["SalePrice"])

# Set up training and testing data
x_train,x_test,y_train,y_test=train_test_split(features, labels, test_size=0.2, random_state=6)


To remove this warning, run the following python code before doing dataframe operations:

    from distributed import Client

    client = Client()

Data types of partitions are different! Please refer to the troubleshooting section of the Modin documentation to fix this issue.


In [4]:

numerical = x_train.select_dtypes(include='number')
numerical_cols = numerical.columns.to_list()

numeric_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='mean')),
    ('scale', MinMaxScaler())
])


categorical = x_train.select_dtypes(exclude=["number"])
categorical_cols = categorical.columns.to_list()

categorical_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False))
])


full_preprocessor = ColumnTransformer(transformers=[
    ('number', numeric_pipeline, numerical_cols),
    ('category', categorical_pipeline, categorical_cols)
])


In [5]:

full_preprocessor.fit_transform(x_train)


array([[0.        , 0.21917808, 0.07976861, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.20205479, 0.04346229, ..., 0.        , 0.        ,
        0.        ],
       [0.05882353, 0.08219178, 0.04253183, ..., 0.        , 1.        ,
        0.        ],
       ...,
       [0.82352941, 0.        , 0.0034831 , ..., 0.        , 1.        ,
        0.        ],
       [1.        , 0.13356164, 0.05247919, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.32534247, 0.07468781, ..., 0.        , 1.        ,
        0.        ]])

In [6]:

lasso = Lasso(alpha=0.1)

lasso_pipeline = Pipeline(steps=[
    ('preprocess', full_preprocessor),
    ('model', lasso)
])

_ = lasso_pipeline.fit(x_train, y_train)


In [7]:

preds = lasso_pipeline.predict(x_test)


0.8446952956040029

In [8]:

print(mean_absolute_error(y_test, preds))
print(lasso_pipeline.score(x_test, y_test))


17728.251457232396
0.8446952956040029
